# Phạm Quốc Toàn
# 197CT22624

### Câu 1 (2 điểm): Cho một ảnh bất kỳ (tên ảnh do sinh viên tự đặt, ví dụ: `my_image.jpg`) và thực hiện các yêu cầu sau:

* Viết chương trình sử dụng Bilateral filter để làm mịn ảnh. (0.5 điểm)  
* Viết chương trình sử dụng Canny Edge Detection để xác định biên của hình ảnh. (0.5 điểm)  
* Đổi màu ảnh bằng cách hoán đổi kênh màu theo thứ tự (ví dụ: BGR → BRG) và lưu thành tên dạng `[ten_anh]_swapped.jpg`. (0.5 điểm)  
* Chuyển ảnh sang không gian màu YCrCb và tách riêng 3 kênh Y, Cr, Cb, lưu thành ảnh grayscale tương ứng (`[ten_anh]_Y.jpg`, `[ten_anh]_Cr.jpg`, `[ten_anh]_Cb.jpg`). (0.5 điểm)


In [ ]:
import cv2
import os

image_name = 'my_image.jpg'
img = cv2.imread(image_name)

if img is None:
    print("Không tìm thấy ảnh. Kiểm tra lại tên hoặc đường dẫn.")
    exit()


output_dir = 'output_1'
os.makedirs(output_dir, exist_ok=True)


base_name = os.path.splitext(image_name)[0]

bilateral = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
cv2.imwrite(os.path.join(output_dir, f'{base_name}_bilateral.jpg'), bilateral)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=100, threshold2=200)
cv2.imwrite(os.path.join(output_dir, f'{base_name}_edges.jpg'), edges)

swapped = img.copy()
swapped[:, :, [1, 2]] = swapped[:, :, [2, 1]] 
cv2.imwrite(os.path.join(output_dir, f'{base_name}_swapped.jpg'), swapped)

ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
Y, Cr, Cb = cv2.split(ycrcb)

cv2.imwrite(os.path.join(output_dir, f'{base_name}_Y.jpg'), Y)
cv2.imwrite(os.path.join(output_dir, f'{base_name}_Cr.jpg'), Cr)
cv2.imwrite(os.path.join(output_dir, f'{base_name}_Cb.jpg'), Cb)

print("Đã xử lý ảnh xong hãy kiểm tra thư mục output_1")


Đã xử lý ảnh xong hãy kiểm tra thư mục output_1


### Câu 2 (4 điểm) Viết một chương trình Python sử dụng OpenCV để tạo menu tương tác cho phép người dùng chọn các kỹ thuật biến đổi hình học và xử lý ảnh nâng cao từ một danh sách, áp dụng đồng thời cho nhiều ảnh.

### Yêu cầu:

1. Menu gồm:  
* Phóng to ảnh (Zoom bằng resize) (0.5 điểm)  
* Xoay ảnh (góc ngẫu nhiên từ 0–360 độ) (0.5 điểm)  
* Lật ảnh ngang (0.5 điểm)  
* Lật ảnh dọc (0.5 điểm)  
* Cắt ảnh (crop ngẫu nhiên vùng giữa ảnh) (0.5 điểm)  
* Thêm viền (padding màu ngẫu nhiên) (0.5 điểm)

2. Chương trình xử lý đồng thời 3 ảnh bất kỳ do sinh viên tự chọn (có thể chọn bằng đường dẫn file hoặc nhập tên ảnh tùy ý). (0.5 điểm)

3. Phím tương ứng để kích hoạt các phương pháp xử lý:  
* Z: Zoom  
* T: Rotate  
* H: Horizontal Flip  
* V: Vertical Flip  
* C: Crop  
* P: Padding (0.5 điểm)

4. Lưu file kết quả với định dạng: `result_[phương pháp]_[tên ảnh gốc].jpg`  
   Ví dụ: `result_crop_cat.jpg`, `result_rotate_image1.jpg` (0.5 điểm)


In [ ]:
import cv2
import numpy as np
import random
import os
from matplotlib import pyplot as plt
from ipywidgets import interact, Dropdown


OUTPUT_DIR = "output_2"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def create_placeholder_images(image_paths):
    print("Kiểm tra ảnh đầu vào...")
    for i, path in enumerate(image_paths):
        if not os.path.exists(path):
            print(f"Không tìm thấy '{path}'. Đang tạo ảnh mẫu...")
        
            height, width = 300, 400
        
            image = np.zeros((height, width, 3), dtype=np.uint8)
        
            color = [(255,0,0), (0,255,0), (0,0,255)][i % 3]
            image[:] = color
            cv2.imwrite(path, image)
            print(f"Đã tạo và lưu ảnh mẫu tại '{path}'.")


image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]
create_placeholder_images(image_paths)

try:
    original_images = {path: cv2.imread(path) for path in image_paths}

    original_images = {k: v for k, v in original_images.items() if v is not None}
    if not original_images:
        print("LỖI: Không thể đọc bất kỳ ảnh nào. Vui lòng kiểm tra lại đường dẫn và tệp tin.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc ảnh: {e}")

def show_images(images, titles):
    """Hiển thị một danh sách các ảnh với tiêu đề tương ứng."""
    plt.figure(figsize=(15, 5))
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, len(images), i + 1)
    
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis("off")
    plt.tight_layout()
    plt.show()


def zoom_image(img, scale=1.5):
    h, w = img.shape[:2]
    resized_img = cv2.resize(img, (int(w * scale), int(h * scale)))
    return {
        "image": resized_img,
        "filename_suffix": "zoom",
        "message": f"Phóng to với tỷ lệ {scale}x"
    }

def rotate_image(img):
    angle = random.randint(0, 360)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated_img = cv2.warpAffine(img, M, (w, h))
    return {
        "image": rotated_img,
        "filename_suffix": "rotate",
        "message": f"Xoay một góc {angle}°"
    }

def flip_horizontal(img):
    flipped_img = cv2.flip(img, 1)
    return {
        "image": flipped_img,
        "filename_suffix": "hflip",
        "message": "Lật theo chiều ngang"
    }

def flip_vertical(img):
    flipped_img = cv2.flip(img, 0)
    return {
        "image": flipped_img,
        "filename_suffix": "vflip",
        "message": "Lật theo chiều dọc"
    }

def crop_center(img, crop_size_ratio=0.5):
    h, w = img.shape[:2]
    ch, cw = int(h * crop_size_ratio), int(w * crop_size_ratio)
    start_y = h // 2 - ch // 2
    start_x = w // 2 - cw // 2
    cropped_img = img[start_y:start_y + ch, start_x:start_x + cw]
    return {
        "image": cropped_img,
        "filename_suffix": "crop",
        "message": f"Cắt vùng trung tâm với tỷ lệ {crop_size_ratio}"
    }

def add_padding(img, pad_size=30):
    color = [random.randint(0, 255) for _ in range(3)]
    padded_img = cv2.copyMakeBorder(img, pad_size, pad_size, pad_size, pad_size,
    cv2.BORDER_CONSTANT, value=color)
    
    return {
        "image": padded_img,
        "filename_suffix": "padding",
        "message": f"Thêm viền màu BGR {color}"
    }


transformation_functions = {
    'Zoom': zoom_image,
    'Rotate': rotate_image,
    'Horizontal Flip': flip_horizontal,
    'Vertical Flip': flip_vertical,
    'Crop': crop_center,
    'Padding': add_padding,
}

def apply_transformation(method_name):
    if not original_images:
        print("Không có ảnh nào để xử lý")
        return

    print(f"\n Áp dụng '{method_name}' cho {len(original_images)} ảnh ")
    
    results = []
    titles = []
    

    transform_func = transformation_functions.get(method_name)
    if not transform_func:
        print(f"Lỗi: Không tìm thấy phương thức xử lý cho '{method_name}'")
        return


    for path, img in original_images.items():
    
        base_name = os.path.splitext(os.path.basename(path))[0]

        result_data = transform_func(img.copy())
        
        result_image = result_data["image"]
        filename_suffix = result_data["filename_suffix"]
        message = result_data["message"]
    
        output_filename = f"result_{filename_suffix}_{base_name}.jpg"
        full_output_path = os.path.join(OUTPUT_DIR, output_filename)

        cv2.imwrite(full_output_path, result_image)
        
        print(f"Ảnh '{base_name}': {message}. Đã lưu tại: '{full_output_path}'")

        results.append(result_image)
        titles.append(output_filename)


    show_images(results, titles)

dropdown = Dropdown(
    options=list(transformation_functions.keys()),
    description='Chọn xử lý:',
    style={'description_width': 'initial'},
    disabled=False,
)

interact(apply_transformation, method_name=dropdown);

Kiểm tra ảnh đầu vào...


interactive(children=(Dropdown(description='Chọn xử lý:', options=('Zoom', 'Rotate', 'Horizontal Flip', 'Verti…

### Câu 3 (4 điểm) Viết một chương trình Python để xử lý 3 ảnh bất kỳ do sinh viên tự chọn.

* Thêm viền đen 20 pixel cho ảnh đầu tiên. (0.5 điểm)  
* Xoay ảnh thứ hai 45 độ và phóng to 1.5 lần. (0.5 điểm)  
* Tăng kích thước ảnh thứ ba lên 4 lần, sau đó áp dụng Bilateral Filter với tham số tùy chọn. (1.5 điểm)  
* Thay đổi độ sáng và độ tương phản ảnh thứ ba theo công thức:

$$
I_{out}(x, y) = \alpha \cdot I_{in}(x, y) + \beta
$$

Trong đó:  

$$
\alpha \in [0.6, 2.0], \quad \beta \in [-60, 60]
$$

Giá trị đầu ra cần được giới hạn trong khoảng [0, 255] bằng công thức:

$$
I_{out}(x, y) = \text{clip}(I_{out}(x, y), 0, 255)
$$


In [ ]:
import cv2
import numpy as np
import os


OUTPUT_DIR = "output_3"
IMAGE_PATHS = ["image1.jpg", "image2.jpg", "image3.jpg"]
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_placeholder_images(image_paths):
    """
    Kiểm tra và tạo ảnh mẫu nếu chúng không tồn tại để mã có thể chạy ngay.
    """
    print("> Bước 1: Kiểm tra ảnh đầu vào")
    for i, path in enumerate(image_paths):
        if not os.path.exists(path):
            print(f"Không tìm thấy '{path}'. Đang tạo ảnh mẫu...")
            image = np.zeros((300, 400, 3), dtype=np.uint8)
        
            color = [(255, 0, 100), (100, 255, 0), (0, 100, 255)][i % 3]
            image[:] = color
        
            cv2.putText(image, f'Image {i+1}', (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 5)
            cv2.imwrite(path, image)
            print(f"Đã tạo ảnh mẫu '{path}'.")
    print("Tất cả ảnh đầu vào đã sẵn sàng.\n")

def add_black_border(image, size=20):
    """Thêm viền đen với kích thước cho trước vào ảnh."""
    return cv2.copyMakeBorder(image, size, size, size, size,
                              borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0])

def rotate_and_scale(image, angle=45, scale=1.5):
    """Xoay và thay đổi tỷ lệ ảnh."""
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(image, rotation_matrix, (w, h))

def enhance_image(image, resize_factor=4.0, alpha=1.5, beta=30, d=9, sigma_color=75, sigma_space=75):
    """
    Một chuỗi xử lý nâng cao chất lượng ảnh bao gồm:
    1. Phóng to ảnh (Resize).
    2. Lọc mịn bằng Bilateral Filter.
    3. Điều chỉnh độ sáng và tương phản.
    """
    h, w = image.shape[:2]
    new_h, new_w = int(h * resize_factor), int(w * resize_factor)
    resized_img = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    filtered_img = cv2.bilateralFilter(resized_img, d, sigma_color, sigma_space)
    
    adjusted_img = np.clip(alpha * filtered_img.astype(float) + beta, 0, 255).astype(np.uint8)
    
    return adjusted_img


processing_pipeline = [
    {
        'description': "Thêm viền đen 20px",
        'input_path': IMAGE_PATHS[0],
        'output_filename': f"result_padding_{os.path.basename(IMAGE_PATHS[0])}",
        'action': add_black_border,
        'params': {'size': 20}
    },
    {
        'description': "Xoay 45 độ và phóng to 1.5x",
        'input_path': IMAGE_PATHS[1],
        'output_filename': f"result_rotate_zoom_{os.path.basename(IMAGE_PATHS[1])}",
        'action': rotate_and_scale,
        'params': {'angle': 45, 'scale': 1.5}
    },
    {
        'description': "Phóng to 4x, lọc Bilateral, điều chỉnh sáng/tương phản",
        'input_path': IMAGE_PATHS[2],
        'output_filename': f"result_enhance_{os.path.basename(IMAGE_PATHS[2])}",
        'action': enhance_image,
        'params': {'resize_factor': 4.0, 'alpha': 1.5, 'beta': 30}
    }
]

def run_processing_pipeline():

    print("> Bước 2: Bắt đầu xử lý ảnh theo quy trình")
    

    create_placeholder_images(IMAGE_PATHS)

    for i, job in enumerate(processing_pipeline):
        print(f"Công việc {i+1}/{len(processing_pipeline)}: {job['description']} cho ảnh '{job['input_path']}'")
        
        image = cv2.imread(job['input_path'])
        
        if image is None:
            print(f"  ✘ LỖI: Không tìm thấy hoặc không thể đọc ảnh '{job['input_path']}'. Bỏ qua.\n")
            continue
            
        try:
            processed_image = job['action'](image, **job['params'])
            output_path = os.path.join(OUTPUT_DIR, job['output_filename'])
            cv2.imwrite(output_path, processed_image)
            print(f"  ✔ Thành công! Đã lưu kết quả tại: '{output_path}'\n")
            
        except Exception as e:
            print(f"  ✘ LỖI khi đang xử lý: {e}\n")


run_processing_pipeline()

> Bước 2: Bắt đầu xử lý ảnh theo quy trình
> Bước 1: Kiểm tra ảnh đầu vào
Tất cả ảnh đầu vào đã sẵn sàng.

Công việc 1/3: Thêm viền đen 20px cho ảnh 'image1.jpg'
  ✔ Thành công! Đã lưu kết quả tại: 'output_3\result_padding_image1.jpg'

Công việc 2/3: Xoay 45 độ và phóng to 1.5x cho ảnh 'image2.jpg'
  ✔ Thành công! Đã lưu kết quả tại: 'output_3\result_rotate_zoom_image2.jpg'

Công việc 3/3: Phóng to 4x, lọc Bilateral, điều chỉnh sáng/tương phản cho ảnh 'image3.jpg'


# Chúc các bạn thi may mắn và đạt điểm 10.